## Data analysis
https://www.r-graph-gallery.com/101_Manhattan_plot.html

In [ ]:
[global]
# Working directory
parameter: out_dir = path('/home/dc2325/results/pleiotropy/2020-04_bolt/BMI/')
#Output merged summary stats file
parameter: name = 'BMI_summary_stats.txt'
#Ouput Manhattan plot
parameter: manhattan= 'BMI_manhattan'

Merge summary stats files using python

In [ ]:
[step_1 (convert)]
#The stats file contains all the summary statistics derived from the association analysis in BOLT
output: f'{out_dir}/{name}.gz'
python: expand=True
    import glob
    file_list = [f for f in glob.glob("*.snp_stats.bgen.gz")]
    with open('{_output}',"w") as outfile:
        with open(file_list[0]) as f1:
            for line in f1:        #keep the header from file1
            outfile.write(line)

        for x in file_list[1:]:
            with open(x) as f1:
                for line in f1:
                    if not line.startswith("SNP"):
                        outfile.write(line)

First of all set the working directory. This is where the BOLT-LMM association results are located.

In [ ]:
[step_2 (plot)]
output: f'{out_dir}/{manhattan}'
R: expand=True
    install.packages("qqman")
    library(qqman)
    summary_gz=gzfile('{_name}.gz','rt')
    df=read.table('{_name}', header=TRUE,sep = ' ')
    png('{_manhattan}.png', width = 6, height = 4, unit='in', res=300)
    bmi_manhattan <- manhattan(df, chr='CHR', bp='BP', snp='SNP', p='P')
    dev.off()

Install and load required libraries

In [ ]:
#install.packages("reader")
#intall.packages("plyr")
#install.packages("ggplot2")
#library(readr)
#library(plyr)
#library(ggplot2)

Then create a list of files to be combined using R. In this case the summary statistics for the association analysis and then create the dataset by binding files from chr{1:22}. The drawback is that R runs out of memory when doing this

In [ ]:
#file_list = Sys.glob("*.snp_stats.bgen.gz") #one way of creating lists
#file_list = list.files(path=mydir, pattern="*.snp_stats.bgen.gz", full.names=TRUE) #another way of creating lists
#data.list = lapply(file_list, function(x){read.table(file = x,header = TRUE, sep = "\t")})
#data.merged = do.call("rbind", data.list

## Drawing the Manhattan Plot

In [ ]:

# Let's highlight them, with a bit of customization on the plot
#manhattan(gwasResults, highlight = snpsOfInterest)
#Annotate the SNPs of interest
#manhattan(gwasResults, annotatePval = 0.01)
#Make a qqplot
#qq(gwasResults$P)